## Download exchanges

In [ ]:
from exchangeReader import ExchangeReader
exr=ExchangeReader()
exr.downloadExchangeCompanyList("NASDAQ")
# exr.downloadAllExchangeCompanyLists()



## Find buys

In [12]:
from exchangeReader import ExchangeReader
import requests
import json
import pandas as pd

exr=ExchangeReader()
companies_table = exr.loadExchange("NASDAQ")
today = dt.date.today()
three_month_td = dt.timedelta(days=90)
three_months_back = today - three_month_td
from_date = three_months_back.strftime("%Y-%m-%d")
to_date = today.strftime("%Y-%m-%d")

missed_symbols = []
for idx, comp_row in companies.iterrows():
    symbol = comp_row["symbol"]
    stock_url = "https://api.nasdaq.com/api/quote/{}/chart?assetclass=stocks&fromdate={}&todate={}".format(symbol, from_date, to_date)
    resp = requests.get(stock_url, headers=exr.headers)
    if resp.ok:
        stock_response = json.loads(resp.content)
        chart = [chart_el["z"] for chart_el in stock_response["data"]["chart"]]
        pd.DataFrame(chart)
    else:
        missed_symbols.append(comp_row)

len(missed_symbols), len(missed_symbols) / len(companies)


In [2]:
companies_table.iloc[0]

symbol                                                   AACG
name         ATA Creativity Global American Depositary Shares
lastsale                                               $0.935
netchange                                              -0.105
pctchange                                            -10.096%
volume                                                  14608
marketCap                                          29544680.0
country                                                 China
ipoyear                                                   NaN
industry                       Service to the Health Industry
sector                                          Miscellaneous
url                              /market-activity/stocks/aacg
Name: 0, dtype: object

<h2>Run asset allocation experiement</h2>

In [ ]:
rs=["FFFGX"]
rs={"FRESX":{'alloc':.2,'min':2500, 'sharesOwned':0, 'amountSpent':0}, 
    "FEMKX":{'alloc':.05,'min':2500, 'sharesOwned':0, 'amountSpent':0}, 
    "FBGRX":{'alloc':.3,'min':2500, 'sharesOwned':0, 'amountSpent':0}, 
    "FIGFX":{'alloc':.15,'min':2500, 'sharesOwned':0, 'amountSpent':0}}

loadSymbols(rs)
for symbol,pkg in rs.items():
    print "{}: {}".format(symbol, len(pkg['data']))
startDate=max([md['data'].index.min() for md in rs.values()])
print "Earliest data for equities is {}".format(startDate)
months=pd.bdate_range(startDate, dt.date.today(), freq='BMS')
    
#TODO: need to assert we have data for each of these symbols or can run simulation

## Any two of the following three can be specified but not all three
## for now we will assume purchasePerMonth and monthsToBalance are the two specified
## purchasePerMonth is the number of purchases that can be made each month
## monthsToBalance is the number of months by which we want to have reached balanced allocation
purchasePerMonth=3; purchaseDays=[7,15,22]; maxPurchasePerMonth=None; monthsToBalance=3; 
baseInvestPerMonth=1000; balanceInvestPerMonth=0; purchasesTilBalance=0;
assert maxPurchasePerMonth is None and (purchasePerMonth is not None or purchaseDays is not None) and monthsToBalance is not None

## totalAtBalance is the smallest 100% total investment at which each asset class can meet its allocation %
#totalAtBalance=max([(1.0/v['alloc'])*v['min'] for s,v in rs.items()])

## remianingToBalance is the amount this asset class needs to still purchase in order to reach its allocation %
#for s,v in rs.items():
#    v['remianingToBalance']=totalAtBalance*(1.0-v['alloc'])

## amountAtEachPurchaseToBalance is the amount of this equity that needs to be purchased at each interval to reach
## the balanced allocation in the set amount of time
#for s,v in rs.items():
#    v['amountAtEachPurchaseToBalance']=totalAtBalance*(1.0-v['alloc'])

buyIntoEquities(rs, startDate)
purchaseAlloc={equity:item['alloc'] for equity, item in rs.items()}
for month in months:
    yearStart = Timestamp(month).is_year_start
    purchaseDates = [month + DateOffset(days=d) for d in purchaseDays]
    for day in purchaseDates:
        print "Buying for month {} and day {}, is year start: {}".format(month, day, yearStart)
        if dateExists(rs, day):
            if yearStart:
                purchaseAlloc=getPurchaseAlloc(rs, day)
                yearStart=False
            for equity, item in rs.items():
                amount=purchaseAlloc[equity]*float(baseInvestPerMonth)
                print "\t {} => alloc: {} amount: {}".format(equity, purchaseAlloc[equity], amount)
                purchaseEquity(item, day, amount)
        else:
            print "\t That day did not exist in all equities"

## Need to include bond payouts into assets
#downloadSymbols(rs.keys())

